# setup

### install dependencies

In [ ]:
!pip install speechbrain
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 50.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00


In [ ]:
!pip install ffmpeg
!pip install moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=c46248d7a5bf529d5cf55ccdf46a058b115e4c2101fcdb29462b0d332e452f33
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### toolkit

In [ ]:
from speechbrain.pretrained import EncoderDecoderASR
from speechbrain import alignment

In [ ]:
import os
import torchaudio

In [ ]:
from moviepy.editor import VideoFileClip

# helper functions

### video2audio conversion

In [ ]:
def video2audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)

### audio2batch segmentation

In [ ]:
def audio2batch(audio_path, batch_dir_path):
    if not os.path.exists(batch_dir_path):
        os.makedirs(batch_dir_path)

    batch_size_sec = 5

    waveform, sample_rate = torchaudio.load(audio_path)

    duration_sec = waveform.size(1) / sample_rate

    num_batches = int(duration_sec / batch_size_sec)

    for i in range(num_batches):
        start_time_sec = i * batch_size_sec
        end_time_sec = (i + 1) * batch_size_sec

        # extract the current batch from the waveform
        start_sample = int(start_time_sec * sample_rate)
        end_sample = int(end_time_sec * sample_rate)
        batch_waveform = waveform[:, start_sample:end_sample]

        output_file_path = os.path.join(batch_dir_path, f"batch_{i+1}.wav")
        torchaudio.save(output_file_path, batch_waveform, sample_rate)

        print(f'batch {i+1} / {num_batches} saved')


    if num_batches * batch_size_sec < duration_sec:
        start_time_sec = num_batches * batch_size_sec
        end_time_sec = duration_sec

        start_sample = int(start_time_sec * sample_rate)
        end_sample = int(end_time_sec * sample_rate)
        last_batch_waveform = waveform[:, start_sample:end_sample]

        output_file_path = os.path.join(batch_dir_path, f"batch_{num_batches+1}.wav")
        torchaudio.save(output_file_path, last_batch_waveform, sample_rate)
        print('partial_batch saved.')
        
    return

### transcription method

In [ ]:
def transcribe(asr_model, batch_dir_path):
    batch_files = os.listdir(batch_dir_path)

    # sort the batch files 
    batch_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

    transcriptions = []
    for i, batch_file in enumerate(batch_files):
        transcription = asr_model.transcribe_file(os.path.join(batch_dir_path, batch_file))
        transcriptions.append(transcription)
        print(f"batch {i+1} / {len(batch_files)} transcripted.")

    subtitles = " ".join(transcriptions)
    return subtitles

### setup models and paths

In [ ]:
# path to the input WAV file
audio_path = "audio.wav"

# path to the output directory
batch_dir_path = "temp_batches/"

In [ ]:
asr_model_1 = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-transformer-transformerlm-librispeech", 
    savedir="pretrained_models/asr-transformer-transformerlm-librispeech", 
    run_opts={'device':'cuda'}
)

asr_model_2 = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-crdnn-transformerlm-librispeech", 
    savedir="pretrained_models/asr-crdnn-transformerlm-librispeech", 
    run_opts={'device':'cuda'}
)

asr_model_3 = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-crdnn-rnnlm-librispeech",
    savedir="pretrained_models/asr-crdnn-rnnlm-librispeech",
    run_opts={'device':'cuda'}
)

# implementation

### input

In [ ]:
video_path = "video_sample.mp4"
asr_model = asr_model_2

### blackbox

In [ ]:
video2audio(video_path, audio_path)
audio2batch(audio_path, batch_dir_path)

MoviePy - Writing audio in audio.wav


MoviePy - Done.
batch 1 / 11 saved
batch 2 / 11 saved
batch 3 / 11 saved
batch 4 / 11 saved
batch 5 / 11 saved
batch 6 / 11 saved
batch 7 / 11 saved
batch 8 / 11 saved
batch 9 / 11 saved
batch 10 / 11 saved
batch 11 / 11 saved
partial_batch saved.


In [ ]:
subtitles = transcribe(asr_model, batch_dir_path)

  warnings.warn(



batch 1 / 12 transcripted.
batch 2 / 12 transcripted.
batch 3 / 12 transcripted.
batch 4 / 12 transcripted.
batch 5 / 12 transcripted.
batch 6 / 12 transcripted.
batch 7 / 12 transcripted.
batch 8 / 12 transcripted.
batch 9 / 12 transcripted.
batch 10 / 12 transcripted.
batch 11 / 12 transcripted.
batch 12 / 12 transcripted.


### outputs

In [ ]:
print(subtitles)

WELCOME TO THE ENGLISH IN A MINUTE MORE JOY'S GOING TO HARDEN'S AND YOU GOT TO BE AROUND FRIENDS AIN'T FOOD AND MAY BE LISTENED TO MUSED IT BUT DO YOUR PARTIES EVER HAVE THE ANIMAL HARDY ANIMAL HE'S ONE OF THEM ARE YOU AS READY AS I I AM FOR THE WEEK END OH YES I'M GOING TO MY BROTHER'S BIRTH DAY PARTY ON FRIDAY MY COUSIN IS HAVING HER GRADUATION PARTY SATURDAY AND I'M DE LA HAYE OAKYARD HE CALLED SONTAG'S WELL YOU ARE QUITE HARDY ANIMAL HERE THAT'S NOTHING YOU SHOULD HAVE SEEN ME IN COLLEGE A PARTY ANIMAL IS NOT AN ACTUAL ANIMAL IT IS A PERSON WHO REALLY ENJOYS GOING TO PARTIES BOTH OF THEM TIME IS SPENT FINDING OUT WHERE THE LATEST AND BEST PARTY IS GOING TO BE AND THAT'S ENGLISH IN A MINUTE
